In [ ]:
!pip install torch torchvision torchaudio


In [2]:
import os
import re
import threading
from urllib.parse import urlparse
import torch
from flask import Flask, request, jsonify
from transformers import BertTokenizer, BertForSequenceClassification
import requests
import json

app = Flask(__name__)  # Initialize flask app , Using flask as backend




In [ ]:
# Loading BERT model  and tokenizer
model_path = "/content/bert_model2.pth"
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")                        # Using pretrained bert-base-uncased model
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
model.eval()


In [4]:
# Function to extract specific data from url for training

# Bert model is trained on text

def extract_url_components(url):
    parsed = urlparse(url)
    hostname = parsed.hostname
    ip_pattern = r"^\d{1,3}(\.\d{1,3}){3}$"
    is_ip = bool(re.match(ip_pattern, hostname)) if hostname else False
    domain = hostname if not is_ip else hostname
    subdomain = "" if is_ip or not hostname else '.'.join(hostname.split('.')[:-2]) if hostname.count('.') > 1 else ''
    path = parsed.path
    file_ext = os.path.splitext(parsed.path)[1] if '.' in os.path.basename(parsed.path) else ''
    query_params = parsed.query
    input_text = f"Domain: {domain} Subdomain: {subdomain} Path: {path} FileExt: {file_ext} Query: {query_params} IP: {is_ip}"
    return input_text




In [11]:
import threading
from flask import Flask, request, jsonify
import torch

app = Flask(__name__)

# Endpoint for testing
@app.route("/predict", methods=["POST"])
def predict():
    data = request.json["url"]
    input_text = extract_url_components(data)
    inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True, max_length=512)

    with torch.no_grad():
        outputs = model(**inputs)
        prediction = torch.argmax(outputs.logits, dim=1).item()

    return jsonify({"malicious": bool(prediction)})

# Start Flask in a separate thread
def run_flask():
    app.run(debug=True, host="127.0.0.1", port=5000, use_reloader=False)

thread = threading.Thread(target=run_flask)
thread.start()


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


In [15]:
import logging
log = logging.getLogger('werkzeug')
log.setLevel(logging.ERROR)


# List of Testing URLs with expected values
with open("/content/data.txt", "r") as f:
    url_data = json.load(f)


# Sending a Post request to the endpoint to check if url is malicious
api_url = "http://127.0.0.1:5000/predict"
headers = {"Content-Type": "application/json"}

# Counters for accuracy calculation
total = len(url_data)
correct = 0

for entry in url_data:
    data = {"url": entry["url"]}
    response = requests.post(api_url, json=data, headers=headers)      # Getting response from backend
    if response.status_code == 200:
        result = response.json().get("malicious")
        if result:
          predicted = "phishing"
        else:
          predicted =  "legit"

        #print(f"URL: {entry['url']}")
        #print(f"Expected: {entry['expected']}, Predicted: {predicted}")
        #print("--" * 20)

        if predicted == entry["expected"]:  # checking if Model gave correct response and increaing counter
            correct += 1
    else:
        print(f"Failed to get response for {entry['url']}")

# CAccuracy calculation
accuracy = (correct / total) * 100
print(f"Accuracy: {accuracy}%")


Accuracy: 90.2439024390244%
